In [1]:
import os, math
import numpy as np
import pandas as pd
from datetime import datetime
from pprint import pprint
import pickle
import omero
import json

import ezomero
#   https://thejacksonlaboratory.github.io/ezomero/ezomero.html#module-ezomero

In [2]:
fh="OVTMA_MarkerQCAnnotations_From_Pathologist.txt"
metadf = pd.read_csv(fh, sep='\t', low_memory=False)
metadf

,Position,ANNEXINV,ASMA,B2MG,CASP3,CD3,CD4,CD8,CD11C,CD14,...,Ki67,LAG3,LAMBDA,MPO,NAKATPASE,PANCK,PD1,PDL1,S6,Tryptase
0,region_001,Good,Fair,Good,Good,Fair,Poor,Good,Good,Good,...,Good,Fair,Good,Good,Good,Poor,Good,Good,Fair,Good
1,region_002,Fair,Good,Fair,Good,Good,Good,Fair,Good,Good,...,Good,Fair,Good,Good,Excellent,Excellent,Good,Good,Fair,Fair
2,region_003,Fair,Fair,Fair,Fair,Excellent,Good,Poor,Good,Fair,...,Good,Fair,Good,Fair,Good,Good,Fair,Poor,Good,Fair
3,region_004,Good,Fair,Excellent,Good,Good,Fair,Good,Good,Fair,...,Good,Fair,Good,Good,Excellent,Excellent,Good,Fair,Fair,Good
4,region_005,Poor,Excellent,Fair,Fair,Good,Fair,Good,Fair,Fair,...,Good,Good,Fair,Fair,Fair,Good,Good,Fair,Good,Poor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,region_344,Fair,Fair,Poor,Good,Fair,Poor,Good,Fair,Fair,...,Good,Good,Good,Fair,Fair,Good,Good,Good,Good,Good
344,region_345,Good,Fair,Good,Good,Excellent,Good,Good,Fair,Good,...,Excellent,Good,Good,Good,Good,Good,Fair,Good,Good,Good
345,region_346,Fair,Fair,Good,Fair,Excellent,Fair,Good,Good,Good,...,Good,Good,Good,Good,Good,Fair,Good,Good,Fair,Fair
346,region_347,Good,Good,Poor,Good,Fair,Fair,Poor,Good,Fair,...,Good,Good,Good,Fair,Fair,Fair,Fair,Good,Fair,Good


In [19]:
# Settings for OMERO
home = str(Path.home())
config = configparser.RawConfigParser()
config.read(r'{}\Documents\myKeyVariables.cfg'.format(home))   
envVars = dict(config.items('OMERO'))

# Settings for OMERO
user = envVars['user']
password = envVars['pass']
host = envVars['url']
port = 4064 #4080
secure = True #False

conn = ezomero.connect(user, password, 'Goode.E_Lab', host=host, port=port, secure=secure)
ezomero.print_datasets(conn, project=408)
setDs = 712

Projects:
	OVCA_TMA22:	408
Datasets in Project "OVCA_TMA22":
	INCell_MxIF:	712
	WSI_czi:	823


In [17]:
getImageByName = {}
imgs = ezomero.get_image_ids(conn, dataset=setDs)
for im in imgs:
    imgDesc, pix = ezomero.get_image(conn, im, xyzct=False, no_pixels=True)
    imName = imgDesc.getName().replace(".ome.tiff","")
    getImageByName[imName] = im
# getImageByName

In [20]:
header = metadf.columns.values
for index, row in metadf.iterrows():
    dTbl = [["Marker","Quality"]]
    _id = getImageByName["OVCA_TMA22_"+row["Position"]]
    print(_id)
    for c in range(len(header)):
        if c == 0:
            continue
        dTbl.append([header[c],row[c]])
    #print(dTbl)
    #break
    ezomero.post_table(conn, dTbl, "Image", _id, 
    title='Pathologist Observations', headers=True)


6497
6784
6710
6591
6741
6578
6773
6529
6632
6796
6516
6673
6774
6526
6718
6681
6479
6455
6693
6511
6625
6698
6747
6540
6692
6617
6467
6655
6574
6489
6651
6589
6534
6560
6671
6468
6608
6713
6548
6641
6678
6571
6739
6675
6469
6481
6676
6731
6524
6677
6793
6553
6760
6652
6472
6762
6554
6752
6588
6562
6732
6624
6696
6464
6486
6690
6618
6503
6653
6546
6750
6687
6626
6530
6602
6723
6568
6792
6714
6537
6780
6631
6480
6770
6716
6790
6505
6669
6776
6604
6706
6786
6508
6756
6563
6501
6619
6695
6764
6475
6670
6594
6521
6620
6683
6483
6460
6685
6607
6458
6686
6569
6721
6688
6788
6456
6599
6754
6539
6657
6767
6541
6782
6664
6463
6787
6715
6727
6558
6680
6751
6555
6711
6465
6478
6757
6549
6527
6586
6614
6735
6582
6658
6523
6585
6736
6570
6761
6644
6550
6801
6699
6473
6491
6557
6797
6500
6649
6661
6579
6656
6633
6533
6777
6542
6502
6609
6712
6771
6597
6627
6488
6600
6722
6611
6512
6628
6596
6528
6630
6612
6507
6577
6743
6506
6798
6700
6572
6728
6660
6509
6800
6697
6766
6487
6646
6765
6531
6744
6682
